In [31]:
##Open csv file
##csvInput = '../insight_testsuite/tests/test_1/input/h1b_input.csv'
csvInput = '../H1b_Data/H1B_FY_2015.csv'

## Identify the entries that we are interested in by analyzing the title
def indexExtration(titles):
    statusIndexList = []
    occupationIndexList = []
    stateIndexList = []

    for i, elem in enumerate(titles):
        if "STATUS" in elem:
            statusIndexList.append(i)
        elif "SOC" in elem and "NAME" in elem:
            occupationIndexList.append(i)
        elif "WORK" in elem and "STATE" in elem:
            stateIndexList.append(i)
            
    ## Keep the index of the first occurance of specific entries
    return statusIndexList[0],occupationIndexList[0],stateIndexList[0]

## Count occurance of each unique item and return the result with occurance count and percentages
def countOccurance(datalist):
    uniqueItem = list(set(datalist))
    totalOccurance = len(datalist)
    itemCount = []
    for item in uniqueItem:
        singleCount = datalist.count(item);
        itemCount.append((item.strip('"'),singleCount, "{:.1%}".format(singleCount/totalOccurance)))
    return sorted(itemCount, key = lambda x: (-x[1],x[0]))

def outputResult(category, resultList):
    f = open('../output/top_10_%s.txt' % category, 'w')
    f.write("TOP_%s;NUMBER_CERTIFIED_APPLICATIONS;PERCENTAGE\n" % category.upper())
    for t in resultList:
        line = ';'.join(str(x) for x in t)
        f.write(line + '\n')
    f.close()
    
with open(csvInput, 'r') as f:
    #read from csv line by line, rstrip remove '\n' at the end of line
    lines = [line.rstrip() for line in f] 

## Identify the title, format title to be all uppercase
titles = list(map(lambda x:x.upper(),lines[0].split(';')))
lines.pop(0);

## Keep the index of the first occurance of specific entries
statusIndex, occupationIndex, stateIndex = indexExtration(titles)

## Define two new lists to keep track of data we are interested in
occupation = []
state = []

## Keep the relevant data on each line
for line in lines:
    words = line.split(';')#get each item in one line
    ##Only document the relevant data of certified candidates
    if words[statusIndex].upper() == 'CERTIFIED':
        occupation.append(words[occupationIndex])
        state.append(words[stateIndex])

## count occurance and output formatted result by        
stateCount = countOccurance(state)
if len(stateCount) > 10:
    outputResult('states', stateCount[:10])
else:
    outputResult('states', stateCount)

    
occupationCount = countOccurance(occupation)
if len(occupationCount) > 10:
    outputResult('occupations', occupationCount[:10])
else:
    outputResult('occupations', occupationCount)